# 基于MindNLP的Prefix Tuning模型微调


## 模块导入&参数初始化


In [1]:
import os
import mindspore
from mindnlp.transformers import AutoModelForSeq2SeqLM
from mindnlp.peft import get_peft_config, get_peft_model, get_peft_model_state_dict, PrefixTuningConfig, TaskType

from mindnlp.dataset import load_dataset
from mindnlp.core import ops

from mindnlp.transformers import AutoTokenizer
from mindnlp.common.optimization import get_linear_schedule_with_warmup
from tqdm import tqdm

model_name_or_path = "t5-small"
tokenizer_name_or_path = "t5-small"

checkpoint_name = "financial_sentiment_analysis_prefix_tuning_v1.ckpt"
max_length = 128
lr = 1e-2
num_epochs = 5
batch_size = 8

/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/ma-user/anaconda3/envs/MindSpore/lib

## Prefix Tuning参数设置以及配置模型

In [2]:

peft_config = PrefixTuningConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, num_virtual_tokens=20)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

T5ForConditionalGeneration has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`.`PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB
trainable params: 122,880 || all params: 60,629,504 || trainable%: 0.20267360260773368


In [3]:
model

PeftModelForSeq2SeqLM(
  (base_model): T5ForConditionalGeneration(
    (shared): Embedding(32128, 512)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 512)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear (512 -> 512)
                (k): Linear (512 -> 512)
                (v): Linear (512 -> 512)
                (o): Linear (512 -> 512)
                (relative_attention_bias): Embedding(32, 8)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseActDense(
                (wi): Linear (512 -> 2048)
                (wo): Linear (2048 -> 512)
                (dropout): Dropout(p=0.1, inplace=False)
                (act): ReLU()
              )
              (layer_norm): T5LayerNorm()
       

## 金融情感分析

### Financial PhraseBank 是一个金融情感分析数据集，
输入： 一个金融短句（例如，“The company's profit has increased significantly this quarter.”）。

输出： 该短句的情感类别（例如，"Positive"）。

我们采用T5,bart模型，将分类问题看作是序列生成问题。

In [4]:
mindspore.dataset.config.set_seed(123)
# loading dataset
dataset = load_dataset("financial_phrasebank", "sentences_allagree",cache_dir='/home/ma-user/work/financial_phrasebank/')

### 输出标签类别

In [5]:
classes = dataset.source.ds.features["label"].names
classes

['negative', 'neutral', 'positive']

### 训练集和验证集划分

In [6]:
train_dataset, validation_dataset = dataset.shuffle(64).split([0.9, 0.1])

[WARNING] ME(212298:281472899117232,MainProcess):2024-12-07-14:33:59.466.821 [mindspore/dataset/engine/datasets.py:1217] Dataset is shuffled before split.


### 将标签号映射成文本标签 0->negative

In [7]:
def add_text_label(sentence, label):
    return sentence, label, classes[label.item()]

train_dataset = train_dataset.map(add_text_label, ['sentence', 'label'], ['sentence', 'label', 'text_label'])
validation_dataset = validation_dataset.map(add_text_label, ['sentence', 'label'], ['sentence', 'label', 'text_label'])

In [8]:
next(train_dataset.create_dict_iterator())

{'sentence': Tensor(shape=[], dtype=String, value= 'The gross area of the Innova 2 project will be about 10,000 sq m ( 107,600 sq ft ) .'),
 'label': Tensor(shape=[], dtype=Int64, value= 1),
 'text_label': Tensor(shape=[], dtype=String, value= 'neutral')}

## 加载模型tokenizer

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

## 将输入和text_label tokenize化

In [10]:
import numpy as np
from mindnlp.dataset import BaseMapFunction
from threading import Lock
lock = Lock()

class MapFunc(BaseMapFunction):
    def __call__(self, sentence, label, text_label):
        lock.acquire()
        model_inputs = tokenizer(sentence, max_length=max_length, padding="max_length", truncation=True)
        labels = tokenizer(text_label, max_length=2, padding="max_length", truncation=True)
        lock.release()
        labels = labels['input_ids']
        labels = np.where(np.equal(labels, tokenizer.pad_token_id), -100, labels)
        return model_inputs['input_ids'], model_inputs['attention_mask'], labels


def get_dataset(dataset, tokenizer, shuffle=True):
    input_colums=['sentence', 'label', 'text_label']
    output_columns=['input_ids', 'attention_mask', 'labels']
    dataset = dataset.map(MapFunc(input_colums, output_columns),
                          input_colums, output_columns)
    if shuffle:
        dataset = dataset.shuffle(64)
    dataset = dataset.batch(batch_size)
    return dataset

train_dataset = get_dataset(train_dataset, tokenizer)
eval_dataset = get_dataset(validation_dataset, tokenizer, shuffle=False)

In [11]:
next(train_dataset.create_dict_iterator())

{'input_ids': Tensor(shape=[8, 128], dtype=Int64, value=
 [[  37, 8690,  616 ...    0,    0,    0],
  [6851,   48,  993 ...    0,    0,    0],
  [  37, 3334,  349 ...    0,    0,    0],
  ...
  [4980,   15,    9 ...    0,    0,    0],
  [9273, 2699, 3751 ...    0,    0,    0],
  [  37, 8148,  349 ...    0,    0,    0]]),
 'attention_mask': Tensor(shape=[8, 128], dtype=Int64, value=
 [[1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  ...
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0]]),
 'labels': Tensor(shape=[8, 2], dtype=Int64, value=
 [[7163,    1],
  [7163,    1],
  [7163,    1],
  ...
  [7163,    1],
  [1465,    1],
  [7163,    1]])}

## 初始化优化器和学习率策略

In [12]:
from mindnlp.core import optim
# optimizer and lr scheduler
optimizer = optim.AdamW(model.trainable_params(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataset) * num_epochs),
)

## 模型训练和评估
### forward 前向传播 ，然后反向传播更新梯度

In [13]:
from mindnlp.core import value_and_grad
# training and evaluation
def forward_fn(**batch):
    outputs = model(**batch)
    loss = outputs.loss
    return loss

grad_fn = value_and_grad(forward_fn, model.trainable_params())

for epoch in range(num_epochs):
    model.set_train()
    total_loss = 0
    train_total_size = train_dataset.get_dataset_size()
    for step, batch in enumerate(tqdm(train_dataset.create_dict_iterator(), total=train_total_size)):
        optimizer.zero_grad()
        loss = grad_fn(**batch)
        optimizer.step()
        total_loss += loss.float()
        lr_scheduler.step()

    model.set_train(False)
    eval_loss = 0
    eval_preds = []
    eval_total_size = eval_dataset.get_dataset_size()
    for step, batch in enumerate(tqdm(eval_dataset.create_dict_iterator(), total=eval_total_size)):
        with mindspore._no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.float()
        eval_preds.extend(
            tokenizer.batch_decode(ops.argmax(outputs.logits, -1).asnumpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataset)
    eval_ppl = ops.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataset)
    train_ppl = ops.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

100%|██████████| 255/255 [00:50<00:00,  5.05it/s]


100%|██████████| 29/29 [00:02<00:00, 10.57it/s]


epoch=0: train_ppl=Tensor(shape=[], dtype=Float32, value= 3.51754) train_epoch_loss=Tensor(shape=[], dtype=Float32, value= 1.25776) eval_ppl=Tensor(shape=[], dtype=Float32, value= 1.36253) eval_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.309347)


100%|██████████| 29/29 [00:02<00:00, 12.70it/s]


epoch=1: train_ppl=Tensor(shape=[], dtype=Float32, value= 1.40963) train_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.343324) eval_ppl=Tensor(shape=[], dtype=Float32, value= 1.29018) eval_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.254784)


...

epoch=2: train_ppl=Tensor(shape=[], dtype=Float32, value= 1.33625) train_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.289866) eval_ppl=Tensor(shape=[], dtype=Float32, value= 1.26808) eval_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.237502)


100%|██████████| 29/29 [00:02<00:00, 12.31it/s]


epoch=3: train_ppl=Tensor(shape=[], dtype=Float32, value= 1.31144) train_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.271126) eval_ppl=Tensor(shape=[], dtype=Float32, value= 1.26708) eval_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.236717)


100%|██████████| 29/29 [00:02<00:00, 12.74it/s]

epoch=4: train_ppl=Tensor(shape=[], dtype=Float32, value= 1.31062) train_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.270502) eval_ppl=Tensor(shape=[], dtype=Float32, value= 1.26243) eval_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.233042)


## 模型评估

In [14]:
# print accuracy
correct = 0
total = 0

ground_truth = []

for pred, data in zip(eval_preds, validation_dataset.create_dict_iterator(output_numpy=True)):
    true = str(data['text_label'])
    ground_truth.append(true)
    if pred.strip() == true.strip():
        correct += 1
    total += 1
accuracy = correct / total * 100
print(f"{accuracy=} % on the evaluation dataset")
print(f"{eval_preds[:10]=}")
print(f"{ground_truth[:10]=}")

accuracy=83.6283185840708 % on the evaluation dataset
eval_preds[:10]=['neutral', 'neutral', 'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'positive', 'negative', 'neutral']
ground_truth[:10]=['neutral', 'neutral', 'neutral', 'neutral', 'positive', 'neutral', 'neutral', 'positive', 'positive', 'positive']


In [15]:
next(eval_dataset.create_tuple_iterator())

[Tensor(shape=[8, 128], dtype=Int64, value=
 [[   37,   804,   594 ...     0,     0,     0],
  [   37,  3829,    29 ...     0,     0,     0],
  [   37,   349,     3 ...     0,     0,     0],
  ...
  [   94,    19,  1316 ...     0,     0,     0],
  [ 1615,    59, 10006 ...     0,     0,     0],
  [   37,  1170,    13 ...     0,     0,     0]]),
 Tensor(shape=[8, 128], dtype=Int64, value=
 [[1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  ...
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0]]),
 Tensor(shape=[8, 2], dtype=Int64, value=
 [[7163,    1],
  [7163,    1],
  [7163,    1],
  ...
  [7163,    1],
  [7163,    1],
  [1465,    1]])]

## 模型保存

In [16]:
# saving model
peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"
model.save_pretrained(peft_model_id)

In [17]:
ckpt = f"{peft_model_id}/adapter_model.ckpt"
!du -h $ckpt

484K	t5-small_PREFIX_TUNING_SEQ_2_SEQ_LM/adapter_model.ckpt


## 加载保存的模型并推理

In [18]:
from mindnlp.peft import PeftModel, PeftConfig

peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)

In [19]:
model.set_train(False)
example = next(validation_dataset.create_dict_iterator(output_numpy=True))
print("input",example['sentence'])
print("true label",example['text_label'])
inputs = tokenizer(example['text_label'], return_tensors="ms")

with mindspore._no_grad():
    outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
    print(tokenizer.batch_decode(outputs.asnumpy(), skip_special_tokens=True))

input The final price will be specified by 14 May 2010 , the acquiring company said .
true label neutral
['neutral']
